# Cleaning the data and providing a small and easily parsable database.

The database with the full logs is too huge to work on. Most of the data would only be needed for debugging but not for evaluation.
Thus, we create a clean and small database for the further evaluation.

In [10]:
import pandas as pd
from _conf import EXTENDED_BASELINE_DATA, PREPROCESSED_RESULTS, RESULT_FOLDER
from algbench import read_as_pandas

In [11]:
# Read the baseline results for comparison. They also already contain instance data (num_clauses and num_features).
baseline_data = pd.read_json(EXTENDED_BASELINE_DATA)
baseline_data.rename(columns={"sample_size": "baseline_sample_size"}, inplace=True)
baseline_data

,instance_name,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime,baseline_sample_size,path_to_baseline_sample,archive_path
0,calculate,9,15,ACTS-IPOG-FT,t2,1620,9.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
1,lcm,9,16,ACTS-IPOG-FT,t2,485,9.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
2,email,10,17,ACTS-IPOG-FT,t2,514,8.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
3,ChatClient,14,20,ACTS-IPOG-FT,t2,538,10.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
4,toybox_2006-10-31_23-30-06,16,13,ACTS-IPOG-FT,t2,485,13.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
...,...,...,...,...,...,...,...,...,...
1945,freetz,31012,102705,FIDE-YASA,t2_m10_null,900009,NaN,None,./EXTERNAL_INPUT/900_seconds_5_it.zip
1946,freetz,31012,102705,FIDE-YASA,t2_m10_null,900008,NaN,None,./EXTERNAL_INPUT/900_seconds_5_it.zip
1947,freetz,31012,102705,FIDE-YASA,t2_m10_null,900008,NaN,None,./EXTERNAL_INPUT/900_seconds_5_it.zip
1948,freetz,31012,102705,FIDE-YASA,t2_m10_null,900008,NaN,None,./EXTERNAL_INPUT/900_seconds_5_it.zip


In [12]:
# Get the data from the result folder
samplns_data = read_as_pandas(
    RESULT_FOLDER,
    lambda result: {
        "parameters": result["parameters"],
        "initial_sample_path": result["parameters"]["args"]["initial_sample_path"],
        "instance_name": result["parameters"]["args"]["instance_name"],
        "lower_bound": result["result"]["lower_bound"],
        "upper_bound": len(result["result"]["solution"]),
        "iteration_info": result["result"]["iteration_info"],
        # "time_used_by_yasa": result["result"]["time_used_by_yasa"],
        # "timelimit_for_samplns": result["result"]["timelimit_for_samplns"],
        # "samplns_used": result["result"]["samplns_used"],
        "runtime": result["runtime"],
        "time_limit": result["parameters"]["args"]["time_limit"],
    }
    if result.get("result", None) and result.get("result", {}).get("solution", None)
    else None,
)
samplns_data.drop_duplicates(subset=["initial_sample_path"], inplace=True)
samplns_data

,parameters,initial_sample_path,instance_name,lower_bound,upper_bound,iteration_info,runtime,time_limit
0,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/30_1_7_5_sample.csv,DMIE,16.0,16,"[{'nbrhd_tuples': 167, 'nbrhd_confs': 4, 'iter...",654.731990,900
1,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/1_1_7_3_sample.csv,calculate,5.0,5,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 10, 'iter...",0.100199,900
2,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/1_1_7_1_sample.csv,calculate,5.0,5,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 10, 'iter...",0.043678,900
3,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/30_1_7_2_sample.csv,DMIE,16.0,16,"[{'nbrhd_tuples': 201, 'nbrhd_confs': 5, 'iter...",710.428369,900
4,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/40_1_7_3_sample.csv,EMBToolkit,1782.0,1878,"[{'nbrhd_tuples': 249, 'nbrhd_confs': 109, 'it...",925.501414,900
...,...,...,...,...,...,...,...,...
557,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/41_1_6_4_sample.csv,atlas_mips32_4kc,30.0,45,"[{'nbrhd_tuples': 197, 'nbrhd_confs': 1, 'iter...",907.957780,900
558,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/21_1_6_2_sample.csv,soletta_2015-06-26_18-38-56,24.0,24,"[{'nbrhd_tuples': 177, 'nbrhd_confs': 8, 'iter...",123.904896,900
559,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/43_1_6_5_sample.csv,integrator_arm7,23.0,46,"[{'nbrhd_tuples': 0, 'nbrhd_confs': 0, 'iterat...",908.704370,900
560,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/27_1_6_2_sample.csv,uclibc_2020-12-24_11-54-53,365.0,365,"[{'nbrhd_tuples': 240, 'nbrhd_confs': 50, 'ite...",315.813920,900


In [13]:
# Pull the baseline info into the data frame
samplns_data_with_baseline_infos = baseline_data.dropna(
    subset="path_to_baseline_sample"
).merge(
    right=samplns_data,
    left_on="path_to_baseline_sample",
    right_on="initial_sample_path",
    # how="right",  # The data may still contain YASA-samples that are corrupted. Not doing `how=right` will drop them.
    suffixes=("_baseline", "_samplns"),
)
samplns_data_with_baseline_infos.drop(
    columns=["instance_name_baseline", "initial_sample_path"], inplace=True
)
samplns_data_with_baseline_infos.rename(
    columns={"instance_name_samplns": "instance_name"}, inplace=True
)
samplns_data_with_baseline_infos

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,archive_path,parameters,instance_name,lower_bound,upper_bound,iteration_info,runtime_samplns,time_limit
0,22,40,Incling,t2,170,12.0,2023-03-01_13-51-03/8_1_2_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",FameDB,8.0,8,"[{'nbrhd_tuples': 224, 'nbrhd_confs': 11, 'ite...",4.386883,900
1,39,77,Incling,t2,205,22.0,2023-03-01_13-51-03/14_1_2_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",SortingLine,9.0,9,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 16, 'ite...",37.380263,900
2,230,1181,Incling,t2,18088,406.0,2023-03-01_13-51-03/24_1_2_5_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",fiasco_2017-09-26_11-30-56,222.0,225,"[{'nbrhd_tuples': 243, 'nbrhd_confs': 83, 'ite...",923.682804,900
3,9,15,FIDE-YASA,t2_m1_null,210,9.0,2023-03-01_13-51-03/1_1_3_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",calculate,5.0,5,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",0.081578,900
4,9,15,FIDE-YASA,t2_m1_null,173,9.0,2023-03-01_13-51-03/1_1_3_2_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",calculate,5.0,5,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",0.079881,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,1298,3036,FIDE-YASA,t2_m10_null,29460,89.0,2023-03-01_13-51-03/45_1_6_2_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",aaed2000,51.0,65,"[{'nbrhd_tuples': 151, 'nbrhd_confs': 1, 'iter...",930.982131,900
289,1298,3036,FIDE-YASA,t2_m10_null,29036,89.0,2023-03-01_13-51-03/45_1_6_4_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",aaed2000,43.0,68,"[{'nbrhd_tuples': 191, 'nbrhd_confs': 2, 'iter...",934.261318,900
290,1397,15692,FIDE-YASA,t2_m10_null,51298,76.0,2023-03-01_13-51-03/46_1_6_2_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",FreeBSD-8_0_0,24.0,66,"[{'nbrhd_tuples': 93, 'nbrhd_confs': 2, 'itera...",907.047551,900
291,1397,15692,FIDE-YASA,t2_m10_null,53625,76.0,2023-03-01_13-51-03/46_1_6_4_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",FreeBSD-8_0_0,28.0,64,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 1, 'iter...",904.672869,900


In [17]:
# shows first 100 lines of pandas
pd.set_option("display.max_rows", 100)
# Check that we have all results.
samplns_data_with_baseline_infos.groupby(
    ["instance_name", "baseline_alg", "baseline_alg_conf"]
).size()

instance_name                baseline_alg  baseline_alg_conf
APL                          FIDE-YASA     t2_m10_null          1
                                           t2_m1_null           5
APL-Model                    FIDE-YASA     t2_m10_null          2
                                           t2_m1_null           5
BankingSoftware              FIDE-YASA     t2_m10_null          2
                                           t2_m1_null           5
BattleofTanks                FIDE-YASA     t2_m10_null          4
                                           t2_m1_null           5
ChatClient                   FIDE-YASA     t2_m10_null          1
                                           t2_m1_null           5
DMIE                         FIDE-YASA     t2_m10_null          2
                                           t2_m1_null           5
E-Shop                       FIDE-YASA     t2_m1_null           5
EMBToolkit                   FIDE-YASA     t2_m10_null          1
               

In [15]:
# Will stop the notebook if the data is bad
# This will probably trigger because not all baseline algorithms were able to compute samples for all instances. -> Adapt as soon as the experiment finishes.
assert (
    samplns_data_with_baseline_infos.groupby(["instance_name", "baseline_alg"])
    .size()
    .min()
    == 5
)
assert (
    samplns_data_with_baseline_infos.groupby(["instance_name", "baseline_alg"])
    .size()
    .max()
    == 5
)

AssertionError: 

In [ ]:
# Save the cleaned data to a json file
samplns_data_with_baseline_infos.to_json(PREPROCESSED_RESULTS)

In [ ]:
# Check that we can read the data
data = pd.read_json(PREPROCESSED_RESULTS)
data

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,archive_path,parameters,instance_name,lower_bound,upper_bound,iteration_info,runtime_samplns,time_limit
0,22,40,Incling,t2,170,12,2023-03-01_13-51-03/8_1_2_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",FameDB,8,8,"[{'nbrhd_tuples': 224, 'nbrhd_confs': 11, 'ite...",4.386883,900
1,39,77,Incling,t2,205,22,2023-03-01_13-51-03/14_1_2_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",SortingLine,9,9,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 16, 'ite...",37.380263,900
2,230,1181,Incling,t2,18088,406,2023-03-01_13-51-03/24_1_2_5_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",fiasco_2017-09-26_11-30-56,222,225,"[{'nbrhd_tuples': 243, 'nbrhd_confs': 83, 'ite...",923.682804,900
3,9,15,FIDE-YASA,t2_m1_null,210,9,2023-03-01_13-51-03/1_1_3_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",calculate,5,5,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",0.081578,900
4,9,15,FIDE-YASA,t2_m1_null,173,9,2023-03-01_13-51-03/1_1_3_2_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",calculate,5,5,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",0.079881,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,1408,3319,FIDE-YASA,t2_m1_null,4915,77,2023-03-01_13-51-03/47_1_3_5_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,27,43,"[{'nbrhd_tuples': 111, 'nbrhd_confs': 2, 'iter...",901.002605,900
210,28,39,FIDE-YASA,t2_m10_null,332,12,2023-03-01_13-51-03/11_1_6_4_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",TightVNC,8,8,"[{'nbrhd_tuples': 245, 'nbrhd_confs': 8, 'iter...",114.311846,900
211,176,280,FIDE-YASA,t2_m10_null,913,40,2023-03-01_13-51-03/23_1_6_2_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",BankingSoftware,28,29,"[{'nbrhd_tuples': 234, 'nbrhd_confs': 11, 'ite...",935.842585,900
212,334,92,FIDE-YASA,t2_m10_null,1959,18,2023-03-01_13-51-03/29_1_6_2_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",toybox_2020-12-06_00-02-46,7,14,"[{'nbrhd_tuples': 237, 'nbrhd_confs': 1, 'iter...",916.203194,900


* `instance_name`: The instance name.
* `num_vars`: The number of variables in the instance.
* `num_clauses`: The number of clauses in the instance.
* `baseline_alg`: The baseline algorithm that created the sample
* `baseline_alg_conf`: The configuration of the baseline algorithm that created the sample.
* `runtime_baseline`: The runtime of the baseline algorithm that created the sample.
* `baseline_sample_size`: The size of the sample created by the baseline algorithm.
* `path_to_baseline_sample`: The path to the sample created by the baseline algorithm.
* `parameters` The parameters used to run samplns.
* `lower_bound` The lower bound computed by SampLNS.
* `upper_bound` The upper bound computed by SampLNS.
* `iteration_info` Info the extract the progress of SampLNS. The time do not include the baseline algorithm.
* `time_used_by_yasa` The time used by the baseline algorithm.
* `timelimit_for_samplns` The time limit for SampLNS. Check also `parameters` for the full timelimit.
* `runtime_samplns` The runtime of SampLNS without the baseline.
* `time_limit` The overall timelimit.
